In [61]:
import numpy as np
import pandas as pd
import datetime as dt
import itertools
from more_itertools import set_partitions

In [62]:
pd.options.display.max_rows = 12
truck_speed = 60
working_hours = 12
m = 7 # Number of trucks
#get a matrix of distances between locations
df = pd.read_csv('dist_vologda_matrix.csv', sep=',')
df = df.iloc[:, 1:]
df = df / 1000
df

,АЗС №35400 Кичменгский Городок,АЗС №35401 Великий Устюг-1,АЗС №35402 Великий Устюг-2,АЗС №35403 Дунай,АЗС №35404 Нюксеница,АЗС №35405 Борисово-Судское,АЗС №35406 Журавлево,АЗС №35407 Грязовец-1,АЗС №35408 Грязовец-2,АЗС №35409 Савино,...,АЗС №35454 Октябрьский пр.-1 (Вологда),АЗС №35455 Октябрьский пр.-2 (Вологда),АЗС №35456 Маяковская,АЗС №35457 Краснодонцев,АЗС №35458 Северсталь,АЗС №35459 Олимпийская,АЗС №35460 Северное шоссе-1,АЗС №35461 Северное шоссе-2,АЗС № 35462 Аксеново,Нефтебаза
0,0.0000,99.2272,101.8513,139.2135,154.2522,777.2635,102.4328,520.5020,517.8863,266.7014,...,610.3886,610.3028,607.3283,609.0579,611.0852,611.0605,604.2684,605.7555,502.9826,477.9325
1,99.0684,0.0000,4.9465,134.5201,149.5589,748.0318,5.5280,491.2703,488.6546,241.7276,...,581.1570,581.0711,578.0967,579.8263,581.8535,581.8288,575.0368,576.5238,473.7510,448.7008
2,101.8148,5.0688,0.0000,137.2665,152.3053,750.7781,3.0302,494.0166,491.4010,244.4739,...,583.9033,583.8174,580.8430,582.5726,584.5999,584.5752,577.7831,579.2702,476.4973,451.4472
3,139.2135,134.6791,137.3031,0.0000,15.0961,613.5690,137.8846,356.8075,354.1918,107.2648,...,446.6942,446.6083,443.6339,445.3635,447.3908,447.3660,440.5740,442.0610,339.2882,314.2380
4,154.2522,149.7178,152.3419,15.0961,0.0000,613.8947,152.9234,357.1332,354.5176,107.5905,...,447.0199,446.9341,443.9596,445.6892,447.7165,447.6918,440.8997,442.3868,339.6139,314.5638
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,611.7861,582.6174,585.2414,447.9957,448.3214,191.2351,585.8229,181.8523,179.2367,345.2112,...,10.1276,10.0417,7.2967,3.3719,10.8241,0.0000,6.1623,4.6720,107.2118,142.1894
60,605.7962,576.6275,579.2515,442.0057,442.3315,185.0682,579.8330,175.8624,173.2467,339.2213,...,6.8726,6.7867,3.8122,4.8070,7.5691,6.8095,0.0000,1.5045,101.2218,136.1995
61,607.1467,577.9779,580.6020,443.3562,443.6820,186.4186,581.1835,177.2129,174.5972,340.5717,...,8.2230,8.1372,5.1627,3.5396,8.9196,5.5422,1.5229,0.0000,102.5723,137.5500
62,504.6641,475.4954,478.1194,340.8736,341.1994,274.3706,478.7009,74.7303,72.1146,238.0892,...,107.4957,107.4099,104.4354,106.1650,108.1923,108.1676,101.3755,102.8626,0.0000,35.0674


In [63]:
#remove locations, which cannot be served in 12 hours (driver can't get there and come back in 12 hours)

to_remove = ((2 * (df.values[-1,:] / truck_speed) < working_hours) == False).nonzero()[0]
df = df.drop(df.index[to_remove])
df = df.drop(df.columns[to_remove], axis=1)
df = df.reset_index(drop=True)
df

,АЗС №35403 Дунай,АЗС №35404 Нюксеница,АЗС №35405 Борисово-Судское,АЗС №35407 Грязовец-1,АЗС №35408 Грязовец-2,АЗС №35409 Савино,АЗС №35410 Сазоново-1,АЗС №35411 Сазоново-2,АЗС №35412 Вытегра,АЗС №35413 Липник,...,АЗС №35454 Октябрьский пр.-1 (Вологда),АЗС №35455 Октябрьский пр.-2 (Вологда),АЗС №35456 Маяковская,АЗС №35457 Краснодонцев,АЗС №35458 Северсталь,АЗС №35459 Олимпийская,АЗС №35460 Северное шоссе-1,АЗС №35461 Северное шоссе-2,АЗС № 35462 Аксеново,Нефтебаза
0,0.0000,15.0961,613.5690,356.8075,354.1918,107.2648,619.0077,618.5201,629.3182,472.6696,...,446.6942,446.6083,443.6339,445.3635,447.3908,447.3660,440.5740,442.0610,339.2882,314.2380
1,15.0961,0.0000,613.8947,357.1332,354.5176,107.5905,619.3335,618.8458,629.6440,472.9954,...,447.0199,446.9341,443.9596,445.6892,447.7165,447.6918,440.8997,442.3868,339.6139,314.5638
2,615.1980,615.5238,0.0000,349.0547,346.4390,512.4135,142.9597,142.4720,165.8965,193.2966,...,187.4480,187.3622,184.7850,188.3816,183.6881,191.1772,185.1426,186.6296,274.4141,309.3918
3,356.1259,356.4516,349.7479,0.0000,3.7018,253.3414,355.1867,354.6990,373.5276,216.8790,...,182.8731,182.7873,179.8128,181.5424,183.5697,183.5450,176.7529,178.2400,75.4671,45.6714
4,353.5507,353.8764,347.1727,3.7422,0.0000,250.7662,352.6115,352.1238,370.9524,214.3038,...,180.2979,180.2120,177.2376,178.9672,180.9945,180.9698,174.1777,175.6648,72.8919,43.0962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51,447.9957,448.3214,191.2351,181.8523,179.2367,345.2112,196.6738,196.1861,332.9386,176.2900,...,10.1276,10.0417,7.2967,3.3719,10.8241,0.0000,6.1623,4.6720,107.2118,142.1894
52,442.0057,442.3315,185.0682,175.8624,173.2467,339.2213,190.5069,190.0192,326.9487,170.3001,...,6.8726,6.7867,3.8122,4.8070,7.5691,6.8095,0.0000,1.5045,101.2218,136.1995
53,443.3562,443.6820,186.4186,177.2129,174.5972,340.5717,191.8574,191.3697,328.2991,171.6506,...,8.2230,8.1372,5.1627,3.5396,8.9196,5.5422,1.5229,0.0000,102.5723,137.5500
54,340.8736,341.1994,274.3706,74.7303,72.1146,238.0892,279.8093,279.3216,346.5072,189.8586,...,107.4957,107.4099,104.4354,106.1650,108.1923,108.1676,101.3755,102.8626,0.0000,35.0674


In [64]:
#shrink array
df = df.iloc[31:56,31:56]
df = df.reset_index(drop=True)
N = len(df.values[0])
df

,АЗС №35439 Козленская,АЗС №35440 Мудрова,АЗС №35441 Преображенского,АЗС №35442 Гагарина (Вологда),АЗС №35443 Маршала Конева-1,АЗС №35444 Маршала Конева-2,АЗС №35445 Чернышевского-1,АЗС №35446 Чернышевского-2,АЗС №35447 Ленинградская,АЗС №35448 Водников,...,АЗС №35454 Октябрьский пр.-1 (Вологда),АЗС №35455 Октябрьский пр.-2 (Вологда),АЗС №35456 Маяковская,АЗС №35457 Краснодонцев,АЗС №35458 Северсталь,АЗС №35459 Олимпийская,АЗС №35460 Северное шоссе-1,АЗС №35461 Северное шоссе-2,АЗС № 35462 Аксеново,Нефтебаза
0,0.0000,7.1360,8.0952,7.7586,2.8882,3.6524,7.4426,6.5003,7.2102,4.8840,...,141.5709,141.4850,138.5106,140.2402,142.2675,142.2428,135.4507,136.9378,34.1649,1.7317
1,7.2492,0.0000,9.2443,8.9077,9.0082,9.7723,4.8814,3.9391,8.3593,4.0128,...,142.7200,142.6341,139.6597,141.3893,143.4166,143.3919,136.5998,138.0869,35.3140,8.5242
2,7.7952,8.8818,0.0000,1.9541,8.2500,7.3115,9.1884,8.2461,2.5565,8.2236,...,133.6286,133.5428,130.5683,132.2979,134.3252,134.3005,127.5084,128.9955,26.2226,9.0701
3,7.8283,8.9149,2.1289,0.0000,6.5645,5.6260,9.2215,8.2791,0.8710,8.2566,...,135.2024,135.1165,132.1421,133.8717,135.8990,135.8742,129.0822,130.5692,27.7964,9.1032
4,2.6551,9.0162,8.1398,6.2794,0.0000,0.8744,9.3228,8.3805,6.4668,6.7642,...,141.2132,141.1273,138.1529,139.8825,141.9098,141.8851,135.0930,136.5801,33.8072,4.3326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,140.9145,142.0011,133.5778,134.7591,141.0550,140.1165,142.3077,141.3654,135.3616,141.3429,...,10.1276,10.0417,7.2967,3.3719,10.8241,0.0000,6.1623,4.6720,107.2118,142.1894
21,134.9246,136.0112,127.5879,128.7692,135.0651,134.1266,136.3178,135.3754,129.3717,135.3530,...,6.8726,6.7867,3.8122,4.8070,7.5691,6.8095,0.0000,1.5045,101.2218,136.1995
22,136.2750,137.3617,128.9384,130.1197,136.4156,135.4771,137.6683,136.7259,130.7221,136.7034,...,8.2230,8.1372,5.1627,3.5396,8.9196,5.5422,1.5229,0.0000,102.5723,137.5500
23,33.7925,34.8791,26.4558,27.6371,33.9330,32.9945,35.1857,34.2433,28.2396,34.2209,...,107.4957,107.4099,104.4354,106.1650,108.1923,108.1676,101.3755,102.8626,0.0000,35.0674


In [65]:
dest = np.arange(N - 1)
dest

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23])

In [66]:
import math
math.factorial(N-1)

620448401733239439360000

In [67]:
def GetLength(route):
    length = df.values[-1, route[0]]
    for i in range(1, len(route)):
        length += df.values[route[i-1], route[i]]
    length += df.values[route[-1], -1]
    
    return length

In [68]:
from itertools import combinations
combs = [np.split(dest, idx) for n_splits in range(m) for idx in combinations(range(1, len(dest)), n_splits)]
print(combs[7])
print("\n")
print(combs[-1])

[array([0, 1, 2, 3, 4, 5, 6]), array([ 7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23])]


[array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17]), array([18]), array([19]), array([20]), array([21]), array([22]), array([23])]


In [69]:
to_del = []
for i in range(len(combs)):
    for j in range(len(combs[i])):
        length = GetLength(combs[i][j])
        if (length / truck_speed) > working_hours:
            to_del.append(i)
len(to_del)

0

In [70]:
new_combs = []
for i in range(len(combs)):
    if i not in to_del:
        new_combs.append(combs[i])
len(new_combs)

145499

In [71]:
max = 0
for comb in new_combs:
    if len(comb) > max:
        max = len(comb)
max

7

In [72]:
shortest = 0
short_dist = working_hours * truck_speed
for i in range(len(new_combs)):
    dist = 0
    for j in range(len(new_combs[i])):
        dist += GetLength(new_combs[i][j])
    if dist < short_dist:
        short_dist = dist
        shortest = i
print("Shortest solution: ", new_combs[shortest])
print("It's total distance is ", short_dist)

Shortest solution:  [array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23])]
It's total distance is  406.8252


In [75]:
for i in range(len(new_combs[shortest])):
    print(GetLength(new_combs[shortest][i]) / truck_speed)

6.78042
